# View management

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 1: Create view with no params

In [ ]:
%%luminesce

-- ==============================================================
-- Description:
-- 1. This query shows you how to make a view
-- 2. The fiew is called "TestHoldings" and will appear under "Test/Example"
-- 3. Luminesce views are effectively custom providers
-- 4. You can think of them as being somewhat like stored procs
-- ==============================================================

-- First delete the view if it already exists

@delete_model_portfolios_view =

use Sys.Admin.SetupView
--provider=Test.Example.TestHoldings
--deleteProvider

----

select 1 as deleting

enduse;

-- Wait 5 seconds after delete before attempting to re-create
-- This give the view time to reset on the Luminesce grid

@delete_model_portfolios_view_response =

select *
from @delete_model_portfolios_view wait 5;

-- The Sys.Admin.SetupView provider

@model_portfolios_view = use Sys.Admin.SetupView
--provider=Test.Example.TestHoldings

----

@model_portfolios = use Drive.csv
--file=/luminesce-examples/model_portfolios.csv
enduse;

select
    #distinct
  #select_agg
{
    {ModelPortfolioName^ : model_port_name},
    {Sector: sect},
    {Region: loc},
    {Weighting: weighting}
}

from @model_portfolios
where #restrict_agg

enduse;

select * from @model_portfolios_view;


#### Step 2: Fetch sql used to create view

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. This query shows the query which was run to create a view
-- ============================================================

select
   Sql, At
from
   sys.logs.hcquery
where
   Sql like '%SetupView%'
   and Sql like '%Test.Example.TestHoldings%'
   and Sql not like '%HcQuery%'
   and ShowAll = true
order by At desc
limit 1


#### Step 3: Current content of view

In [ ]:
%%luminesce

-- =========================================================
--  Description:
--  1. This query shows you the contents of a view
--  2. The results alone cannot be used to recreate the view
--  3. You will need to query sys.logs.hcquery to get the SQL
--  used to create  a view
-- =========================================================

select
  Content
from
  sys.file
where
  directory = 'databaseproviders/Test/Example'
and
  name = 'TestHoldings'


#### Step 4: Show view history

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- This query shows the history of changes to a view
-- (essentially SCCM for views)
-- ============================================================

select
  *
from
  sys.file.history
where
  directory = 'databaseproviders/Test/Example'
and
  name = 'TestHoldings'


#### Step 5: Show view dependencies

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- This query shows what deoendencies a view has, including other views and data providers.
-- "includeindirect" means the dependencies are fully expanded all the way to the data providers 
-- ============================================================

select
  *
from
  sys.dependency
where
  name = 'Test.Example.Testholdings'
and
  includeindirect = true
and
  showall = true



#### Step 6: Show dependencies on view

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- This query shows which views depending on a particular view
-- ============================================================

select
  *
from
  sys.dependency
where
  dependency = 'Drive.Csv'
and
  includeindirect = true
and
  showall = true



#### Step 7: Delete a view

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- This query shows you how to delete a custom view
-- Wait 5 seconds after delete before attempting to re-create
-- This give the view time to reset on the Luminesce grid
-- ============================================================

@delete_model_portfolios_view = use Sys.Admin.SetupView
--provider=Test.Example.TestHoldings
--deleteProvider
----

select 1 as deleting_view

enduse;

select * from @delete_model_portfolios_view;


#### Step 8: Run table of query strings

In [ ]:
%%luminesce

/*

-- Description:
-- Run a table of query strings

===========================================
    1. Create a view to run query strings 
===========================================
*/


@sqlRunner = use Sys.Admin.SetupView 
--provider=AdHocQuery.Runner
--parameters   
sqlString,Text,select * from sys.field limit 2;,true
----

@@sep = select '--' || '--';
@@sql = select #PARAMETERVALUE(sqlString);

@x = use Sys.Admin.SetupView with @@sql, @@sep
{@@sep}
{@@sql}

enduse;

select 'Done' as Result;

enduse;

@createQueryView = select * from @sqlRunner; 

/*
=================================
    2. Define a set of queries 
=================================

    In the example below, we have defined the query strings directly in this script.
    In practice, you might want to load these from another location such as the LUSID
    config store.

*/

@sqlStrings = select 
'select * from lusid.instrument limit 5' as 'sql_strings'
union all
values
('select * from sys.field limit 10'),
('select LusidInstrumentId, CouponRate from Lusid.Instrument.Bond limit 10');

/*
===================================
    3. Run the queries
===================================
*/

select s.sql_strings, results.*
from @sqlStrings s
outer apply (
        select a.^
        from Adhocquery.Runner a
        where a.sqlString = s.sql_strings
) results;

